In [20]:
from PIL import Image
import json

In [60]:
from torchvision import models, transforms
from torch.autograd import Variable
import torch.nn as nn

In [2]:
alexnet = models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /Users/zui/.torch/models/alexnet-owt-4df8aa71.pth
98.5%

In [3]:
alexnet

AlexNet (
  (features): Sequential (
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU (inplace)
    (2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU (inplace)
    (5): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU (inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU (inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  )
  (classifier): Sequential (
    (0): Dropout (p = 0.5)
    (1): Linear (9216 -> 4096)
    (2): ReLU (inplace)
    (3): Dropout (p = 0.5)
    (4): Linear (4096 -> 4096)
    (5): ReLU (inplace)
    (6): Linear (4096 -> 1000)
  )
)

In [19]:
def load_labels(label_path="imagenet_labels.json"):
    with open("imagenet_labels.json", "r") as f:
        labels = f.read()
    labels = json.loads(labels)
    return labels

In [4]:
def prepare_image(image):
    """
    Prepare image to be fed into the model.
    
    
    """
    
    # These preprocessing steps are taken from PyTorch's ImageNet example
    preprocess = transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
           mean=[0.485, 0.456, 0.406],
           std=[0.229, 0.224, 0.225]
        ),
    ])
    return preprocess(image).unsqueeze_(0)

In [46]:
def top_n_prediction(result, n=10, labels=labels):
    """
    Get top n prediction from the result and return those as labels
    
    :param Variable result: Result obtain at the other end of the classifier
    :param int n: top n label
    :param dict labels: labels dictionary
    """
    
    top_n = result.data.numpy().argsort()[0][::-1][:n]
    
    return [labels[str(x)] for x in top_n]
    
    

In [42]:
top_n_prediction(Out[12], 10, labels)

['folding chair',
 'rocking chair, rocker',
 'dining table, board',
 'bassinet',
 'throne',
 'plate rack',
 'cradle',
 'desk',
 'crib, cot',
 'guillotine']

In [43]:
wine_g = Variable(prepare_image(Image.open("wine_glass.jpg")))

Variable containing:
( 0 , 0 ,.,.) = 
  2.2489  2.2489  2.2489  ...   2.2489  2.2489  2.2489
  2.2489  2.2489  2.2489  ...   2.2489  2.2489  2.2489
  2.2489  2.2489  2.2489  ...   2.2489  2.2489  2.2489
           ...             ⋱             ...          
  2.2489  2.2489  2.2489  ...   2.2489  2.2489  2.2489
  2.2489  2.2489  2.2489  ...   2.2489  2.2489  2.2489
  2.2489  2.2489  2.2489  ...   2.2489  2.2489  2.2489

( 0 , 1 ,.,.) = 
  2.4286  2.4286  2.4286  ...   2.4286  2.4286  2.4286
  2.4286  2.4286  2.4286  ...   2.4286  2.4286  2.4286
  2.4286  2.4286  2.4286  ...   2.4286  2.4286  2.4286
           ...             ⋱             ...          
  2.4286  2.4286  2.4286  ...   2.4286  2.4286  2.4286
  2.4286  2.4286  2.4286  ...   2.4286  2.4286  2.4286
  2.4286  2.4286  2.4286  ...   2.4286  2.4286  2.4286

( 0 , 2 ,.,.) = 
  2.6400  2.6400  2.6400  ...   2.6400  2.6400  2.6400
  2.6400  2.6400  2.6400  ...   2.6400  2.6400  2.6400
  2.6400  2.6400  2.6400  ...   2.6400  2.6400

In [45]:
wine_g_pred = alexnet(wine_g)

In [47]:
top_n_prediction(wine_g_pred)

['red wine',
 'goblet',
 'eggnog',
 'wine bottle',
 'punching bag, punch bag, punching ball, punchball',
 'cocktail shaker',
 'beer glass',
 'table lamp',
 'swab, swob, mop',
 'hourglass']

In [48]:
durian = Variable(prepare_image(Image.open("durian.png")))

In [50]:
durian_pred = alexnet(durian)

In [52]:
top_n_prediction(durian_pred)

['daisy',
 'buckeye, horse chestnut, conker',
 'broccoli',
 'jackfruit, jak, jack',
 'acorn',
 'stole',
 'pot, flowerpot',
 'coil, spiral, volute, whorl, helix',
 'custard apple',
 'cardoon']

In [56]:
for p in alexnet.parameters():
    # We freeze all the parameters in the model.
    p.requires_grad = False

In [57]:
alexnet.fc.in_features

AttributeError: 'AlexNet' object has no attribute 'fc'

In [61]:
model_conv = models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

In [62]:
num_ftrs

512

In [73]:
model_conv

ResNet (
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU (inplace)
  (maxpool): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential (
    (0): BasicBlock (
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock (
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d

In [64]:
model_conv.fc.in_features

512

In [72]:
alexnet.classifier[6].in_features

4096

In [ ]:
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
squeezenet = models.squeezenet1_0(pretrained=True)
vgg16 = models.vgg16(pretrained=True)
densenet = models.densenet161(pretrained=True)
inception = models.inception_v3(pretrained=True)